In [1]:
import json
import math
import re
import time
from collections import Counter
from urllib.request import urlopen

from bs4 import BeautifulSoup
import requests

In [2]:
def getPageNumber(href):
    res = requests.get(href)
    soup = BeautifulSoup(res.text, 'lxml')
    numberReg = re.compile(r'(\d+ )筆')
    mo = numberReg.search(soup.select('ul > li.right')[0].text)
    pageNum = int(mo.group(1))
    totalPages = 150 if math.ceil(pageNum / 20) > 150 else math.ceil(pageNum / 20) # 除以20筆得出幾頁
    res.close()
    return totalPages

In [3]:
def getPageLinks(href):
    linkList = []
    res = requests.get(href)
    soup = BeautifulSoup(res.text, 'lxml')
    # 找出所有頁面上的連結，加入list
    jobList = soup.select("div.jobname_summary > a")
    for job in jobList:
        link = "https://www.104.com.tw" + job["href"]
        linkList.append(link)
    res.close()
    return linkList
def getJobInfo(href):
    jobInfo = dict()
    res = requests.get(href)
    soup = BeautifulSoup(res.text, 'lxml')
    jobInfo["title"] = soup.select_one("header > div > h1").contents[0].replace("\r\n", "").strip(" ")
    jobInfo["company"] = soup.select_one("a.cn").text
    jobInfo["content"] = soup.select_one("div.content > p").text.replace("\r", "").replace(" ", "")
    jobInfo["tools"] = soup.select("dd.tool")[0].text
    jobInfo["skills"] = soup.select("dd.tool")[1].text
    jobInfo["post_date"] = soup.select_one("time.update").text.split("：")[1]
    jobInfo["other_condition"] = soup.select("section.info")[1].select("dl > dd ")[-1].text.replace("\r","").replace(" ","")
    res.close()
    return jobInfo
def saveData(data,fileName = "data.json"):
    # 輸出成json檔
    with open(fileName, 'w') as f:
        json.dump(data, f)
if __name__ == "__main__":
    data = []
    href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=軟體設計工程師&jobsource=n104bank1&jobcat=2007001001&order=1&page="
    pageNum = getPageNumber(href)
    print(pageNum)
    num = 1
    for n in range(0,pageNum+1):
        try:
            web = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=軟體設計工程師&jobsource=n104bank1&jobcat=2007001001&order=1&page={}".format(num)
            jobLink=getPageLinks(web)
            for link in jobLink:
                jobInfo = getJobInfo(link)
                data.append(jobInfo)
                time.sleep(5)
            num +=1
        except Exception as e:
            print(e)
            pass
    saveData(data,"104_Get_軟體設計工程師_cat1001.json")

40
HTTPSConnectionPool(host='www.104.com.twhttp', port=443): Max retries exceeded with url: //hunter.104.com.tw/104hunter.jsp?trc=104p&jobsource=n104bank1&urlpage=job_description&jobno=9173566 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x00000000085379B0>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))
HTTPSConnectionPool(host='www.104.com.twhttp', port=443): Max retries exceeded with url: //hunter.104.com.tw/104hunter.jsp?trc=104p&jobsource=n104bank1&urlpage=job_description&jobno=9173566 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x0000000007A759B0>: Failed to establish a new connection: [Errno 11004] getaddrinfo failed',))
HTTPSConnectionPool(host='www.104.com.twhttp', port=443): Max retries exceeded with url: //hunter.104.com.tw/104hunter.jsp?trc=104p&jobsource=n104bank1&urlpage=job_description&jobno=9173566 (Caused by NewConnectionError